In [1]:
import pandas as pd

In [6]:
df1 = pd.read_csv('train1.csv')
df0 = pd.read_csv('test1.csv')

In [7]:
df1.shape,df0.shape

((7920, 3), (1953, 2))

In [4]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [9]:
df1.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [10]:
messages1 = df1.copy()
messages0 = df0.copy()

In [15]:
# stemming train dataset
ps = PorterStemmer()
corpus1 = []
for i in range(0, len(messages1)):
    review1 = re.sub('[^a-zA-Z]', ' ', messages1['tweet'][i])
    review1 = review1.lower()
    review1 = review1.split()
    
    review1 = [ps.stem(word1) for word1 in review1 if not word1 in stopwords.words('english')]
    review1 = ' '.join(review1)
    corpus1.append(review1)

In [16]:
# stemming test dataset
corpus0 = []
for i in range(0, len(messages0)):
    review0 = re.sub('[^a-zA-Z]', ' ', messages0['tweet'][i])
    review0 = review0.lower()
    review0 = review0.split()
    
    review0 = [ps.stem(word0) for word0 in review0 if not word0 in stopwords.words('english')]
    review0 = ' '.join(review0)
    corpus0.append(review0)

In [80]:
# bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 7300)
X = cv.fit_transform(corpus1).toarray()
test0 = cv.fit_transform(corpus0).toarray()

In [81]:
test0

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [82]:
Y = messages1['label'].values

In [83]:
# train-test split
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(X, Y , test_size = 0.20 ,random_state = 0)

In [84]:
from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB().fit(x_train , y_train)
ypred1 = model1.predict(x_test)

In [85]:
ypred0 = model1.predict(test0)

In [86]:
print (ypred0)

[1 0 1 ... 1 1 1]


In [87]:
output_df = pd.DataFrame({'id' : messages0['id'].values })
output_df['label'] = ypred0

In [91]:
output_df.to_csv('submission3.csv' , index = False)

In [89]:
rf_accuracy1 = round(model1.score(x_train,y_train),2)
rf_accuracy1

0.91

In [90]:
rf_accuracy2 = round(model1.score(x_test,y_test),2)
rf_accuracy2

0.89